In [2]:
from twitter.twitter import *
import os,sys,time
import datetime
import locale
import auth_info as auth

locale.setlocale(locale.LC_ALL, 'ja_JP.utf8')

MAX_RECURSIVE_DEPTH = 20

#NG words
#ここでタプルに追加した単語（記号）を含むツイートは収集から除外される。
#今回は、URLやHashtag、【定期】のように使われるカッコなどを除外とした。
check_chara = ('http', '#', '\\', '【','】')

#Retry MAX
retry_max = 10
#Retry time
retry_time = 10

#過去に認証したときのTokenを探す。
TWITTER_CREDS = os.path.expanduser('.credentials')
if not os.path.exists(TWITTER_CREDS):
  #無ければ、OAuthで認証
  oauth_dance(auth.CONSUMER_NAME, auth.CONSUMER_KEY, auth.CONSUMER_SECRET, TWITTER_CREDS)
oauth_token, oauth_secret = read_token_file(TWITTER_CREDS)

# token
# Tokenを用いて、Streaming APIで新着ツイートを見つけるstreamと、普通のTwitter APIで'in_reply_to_status_id'を参照するtwitterの2つを用意する。
stream = TwitterStream(auth=OAuth(oauth_token, oauth_secret, auth.CONSUMER_KEY, auth.CONSUMER_SECRET))
twitter = Twitter(auth=OAuth(oauth_token, oauth_secret, auth.CONSUMER_KEY, auth.CONSUMER_SECRET))

In [3]:
def fwrite(fname, str):
    f = open(fname, 'a')
    f.write(str + '\n')
    f.close()

In [4]:
#in_reply_to_status_idから宛先となるツイートを参照する。
def show(_status):
  global current_recursive_depth, data_num
  #print(current_recursive_depth)
  #print('IN:\n'+str(_status), file=sys.stderr)
  #リトライを設定
  #print (_status['text'])
  for r in range(retry_max):
    #言語が日本語、in_reply_to_status_idが明示されており、禁止単語を含まないツイートの場合
    if 'lang' in _status and _status['lang'] == 'ja' and 'in_reply_to_status_id' in _status and not _status['in_reply_to_status_id'] is None and 'text' in _status and check(_status['text']):
      try:
        #ツイートを参照
        status=twitter.statuses.show(id=_status['in_reply_to_status_id'])
      except:
        #print('! Error\tRetry to read in_reply_tweet - '+str(r), file=sys.stderr)
        time.sleep(retry_time)
        continue
      #print('OUT:\n'+str(status), file=sys.stderr)
      #宛先のツイートが日本語、in_reply_to_status_idが明示されており、禁止単語を含まないツイートの場合
      if 'lang' in _status and _status['lang'] == 'ja' and 'text' in status and check(status['text']):
        #標準出力で応答の対を出力
        #print('--------------------------------------------------------')
        #print('tweet:\t'+str(_status['id'])+'\t'+trim(_status['text']))
        #print('reply:\t'+str(status['id'])+'\t'+trim(status['text']))
        #さらに、宛先のツイートがin_reply_to_status_idを持つ場合は、さらに参照を行う。
        fwrite('data_in.txt', trim(_status['text']))
        fwrite('data_out.txt', trim(status['text']))
        data_num += 1
        if data_num % 10 == 0 :
          d = datetime.datetime.today()
          fwrite('data_ins_log.txt', '%s #%d' % (d.strftime("%x %X"), data_num))
          print('%s #%d' % (d.strftime("%x %X"), data_num))
        if 'in_reply_to_status_id' in status and not status['in_reply_to_status_id'] is None and current_recursive_depth <= MAX_RECURSIVE_DEPTH :
          current_recursive_depth += 1
          show(status)
          current_recursive_depth -= 1
        break
      else:
        #print('! Info\tin_reply_tweet is not acceptable', file=sys.stderr)
        break
    else:
      break

#改行を適当に置き換える
def trim(text):
  return text.replace('\r',' ').replace('\n',' ')

#禁止文字のチェックを行う
def check(text):
  #for char in check_chara:
  #  if char in text:
  #    return False
  return True

In [ ]:
current_recursive_depth = 0
data_num = 0
while 1:
  #ストリームに接続して適当にツイートを読み出す
  statuses = stream.statuses.sample()
  for status in statuses:
    #参照に送る
    show(status)

In [15]:
d = datetime.datetime.today()

In [17]:
d = datetime.datetime.today()
print "%s %d" % (d.strftime("%x %X"), 3)

2016年10月12日 23時30分30秒 3


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe5 in position 4: ordinal not in range(128)